In [12]:
import os
import asyncio
import httpx
from datetime import datetime
from typing import List, Optional, Union
from pydantic import BaseModel


# =========================
# Data model (Search only)
# =========================

class SearchResult(BaseModel):
    url: str
    title: Optional[str] = None
    snippet: Optional[str] = None
    engine: Optional[str] = None
    score: Optional[float] = None
    extracted_at: str


class SearchError(BaseModel):
    query: str
    error: str
    extracted_at: str
    success: bool = False


# =========================
# SearxNG Search Step 1
# =========================

class SearxNGSearch:
    def __init__(self, base_url: Optional[str] = None):
        self.base_url = base_url or os.getenv("SEARXNG_BASE_URL", "http://localhost:8888/search")
        if not self.base_url:
            raise ValueError("SEARXNG_BASE_URL is not set")

    async def _fetch_page(
        self,
        client: httpx.AsyncClient,
        query: str,
        page: int,
        timeout: int,
    ) -> Union[List[dict], str]:
        try:
            params = {
                "q": query,
                "format": "json",
                "pageno": page,
            }
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
                "Accept": "application/json",
            }
            r = await client.get(self.base_url, params=params, timeout=timeout, headers=headers)
            r.raise_for_status()
            return r.json().get("results", [])
        except Exception as e:
            return f"Page {page} failed: {e}"

    async def _search_async(
        self,
        query: str,
        max_pages: int,
        timeout: int,
        max_results: Optional[int],
        output_for_llm: bool,
    ) -> Union[List[SearchResult], str, SearchError]:

        collected: List[SearchResult] = []

        async with httpx.AsyncClient() as client:
            tasks = [
                self._fetch_page(client, query, page, timeout)
                for page in range(1, max_pages + 1)
            ]
            pages = await asyncio.gather(*tasks)

        for page in pages:
            if isinstance(page, str):
                return SearchError(
                    query=query,
                    error=page,
                    extracted_at=datetime.now().isoformat(),
                )

            for item in page:
                collected.append(
                    SearchResult(
                        url=item.get("url"),
                        title=item.get("title"),
                        snippet=item.get("content"),  # SearxNG snippet
                        engine=item.get("engine"),
                        score=item.get("score"),
                        extracted_at=datetime.now().isoformat(),
                    )
                )

        if max_results:
            collected.sort(
                key=lambda x: x.score if x.score is not None else 0,
                reverse=True,
            )
            collected = collected[:max_results]

        if output_for_llm:
            return "\n".join(r.model_dump_json() for r in collected)

        return collected

    async def asearch(
        self,
        query: str,
        *,
        max_pages: int = 3,
        timeout: int = 10,
        max_results: Optional[int] = None,
        output_for_llm: bool = False,
    ):
        return await self._search_async(
            query=query,
            max_pages=max_pages,
            timeout=timeout,
            max_results=max_results,
            output_for_llm=output_for_llm,
        )

    def search(
        self,
        query: str,
        *,
        max_pages: int = 3,
        timeout: int = 10,
        max_results: Optional[int] = None,
        output_for_llm: bool = False,
    ) -> Union[List[SearchResult], str, SearchError]:

        return asyncio.run(
            self._search_async(
                query=query,
                max_pages=max_pages,
                timeout=timeout,
                max_results=max_results,
                output_for_llm=output_for_llm,
            )
        )


# =========================
# Example usage (Step 1)
# =========================

if __name__ == "__main__":
    searcher = SearxNGSearch()

    results = await searcher.asearch(
    query="鹿島建設株主優待の有無・内容",
    max_pages=3,
    max_results=5,
    output_for_llm=False,   # 👈
    )

    content = []
    for result in results:
        content.append(result.snippet)
        print(result)
    text = "\n\n".join(f"- {c.strip()}" for c in content)
    print(text)



url='https://www.kajima.co.jp/ir/faq/?mode=pc' title='よくあるご質問（FAQ） | 鹿島建設株式会社' snippet='現在、株主優待制度は実施しておりません。 ページのトップへ戻る. Question 株主還元方針、配当金の推移について知りたい。 Answer. 配当\xa0...' engine='startpage' score=1.0 extracted_at='2026-01-08T15:08:16.923333'
url='https://www.jreast.co.jp/company/deal/border/' title='土地境界立会いについて｜法人お取引に関して - JR東日本' snippet='事前打ち合わせでは、当社グループ会社等が申請内容の確認を行います。 【当社グループ会社】 ○首都圏本部・横浜支社・八王子支社・大宮支社・高崎支社・水戸支社・千葉\xa0...' engine='startpage' score=1.0 extracted_at='2026-01-08T15:08:16.923577'
url='https://doda.jp/DodaFront/View/JobSearchList/j_oc__020314S/-op__1/-preBtn__3/' title='【知的財産（知財）・特許】新着の転職・求人・中途採用情報 doda ...' snippet='知的財産（知財）・特許、新着の転職・求人情報ページ：公開求人数69件。「完全週休2日制」「職種・業種未経験歓迎」など、条件にあった検索ができます。' engine='startpage' score=1.0 extracted_at='2026-01-08T15:08:16.923639'
url='https://www.kajima.co.jp/ir/faq/index-j.html' title='よくあるご質問（FAQ） | 鹿島建設株式会社' snippet='Q. 株主優待制度の有無について知りたい。 Q. 株主還元方針、配当金の推移について知りたい。' engine='startpage' score=0.5 extracted_at='2026-01-08T15:08:16.923505'
u